## Goal
Here, we will use RESNET34 architecture to train a classification model to identify the age range of images.

Age range definition based on Health Promotion Board of Singapore
- (A0) Young Children: 0-6 years 
- (A1) Children and Youth: 7-17 years 
- (A2) Youth Adults: 18-25 years
- (A3) Adults: 26-49 years
- (A4) Older adults: 50 years

The training data used is from UTKFace.

In [1]:
import math

save_directory = "imgs/cropface/"
# img = cv2.imread(sample_img)
classNames = ["face"]

def cropimgs(results, save_directory, expand_ratio = 0.2, CI = 0.7):
    # results: list of results for all the images after running the model
    # save_directory: location of output directory
    # expand_ratio: expand the size of x1,x2 by expand_ratio percent. 
    # CI: confidence interval of prediction. Only crop the image if the confidence is higher than this threshold
    
    for r in results:
        boxes = r.boxes
        imgpath = r.path
        img = cv2.imread(imgpath)
        
        cropstr = imgpath.split("/")[-1].split(".jpg")[0]
        img_index = 0

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
            
            # Calculate expanded coordinates
            x1 = max(0, int(x1 - expand_ratio * (x2 - x1)))
            y1 = max(0, int(y1 - expand_ratio * (y2 - y1)))
            x2 = min(img.shape[1], int(x2 + expand_ratio * (x2 - x1)))
            y2 = min(img.shape[0], int(y2 + expand_ratio * (y2 - y1)))

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            # print("Confidence --->",confidence)

            # # class name
            # cls = int(box.cls[0])
            # print("Class name -->", classNames[cls])

            if classNames[cls] == "face" and confidence > CI:
                # Crop the person from the image
                cropped_person = img[y1:y2, x1:x2]

                # Save the cropped person image
                filename = f"{save_directory}{cropstr}_{img_index}.jpg"
                cv2.imwrite(filename, cropped_person)
                img_index += 1
